<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Applying AI Weather Models With NVIDIA Earth-2</h1>
<h2><b>Part 1:</b> Forecasting</h2>
<br>

In the first part of our workshop, we will create medium-range weather forecasts using global AI weather models. Weather has a significant impact on various industries. For instance, extreme weather events can disrupt supply chains, energy infrastructure, and pose a risk to the public. Furthermore, weather conditions affect everything from agricultural yields to energy consumption and production. Accurate weather forecasts enable stakeholders to make informed decisions, manage risks, and optimize their strategies. By understanding weather patterns, businesses can better anticipate market fluctuations and protect their investments.

Global AI weather models predict the future state of the atmosphere based on the current state of the atmosphere. Compared to numerical weather models, AI models produce weather forecasts orders of magnitude faster and run on much smaller hardware. The models are trained on large corpora of historic weather data, most notably ERA5, the fifth generation of the ECMWF atmospheric reanalysis. Inference happens through autoregressive rollouts, which means that the model output from the previous time step is fed back into the model to calculate the next time step.

Medium-range forecasts reach 10-14 days into the future, and this is considered to be the time frame where deterministic forecasts are possible. The goal of a deterministic forecast is to predict the single most likely evolution of weather over time. For building reliable weather applications, however, we are generally interested in probabilistic forecasts. A probabilistic forecast can tell us how likely certain weather events or weather conditions are going to occur. While numerical probabilistic forecasts are extremely compute intensive, AI models can produce probabilistic forecasts rapidly, making them highly efficient and accessible for custom applications.

In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

import numpy as np

from earth2studio import run
from earth2studio.data import GFS, NCAR_ERA5
from earth2studio.io import KVBackend
from earth2studio.lexicon import NCAR_ERA5Lexicon
from earth2studio.models.px import SFNO
from earth2studio.perturbation import CorrelatedSphericalGaussian, SphericalGaussian
from earth2studio.utils.time import to_time_array
from hens import HemisphericCentredBredVector, get_noise_vector
from plot import animate_local, figure_global, plot_global_ensemble, plot_perturbations
from utils import get_lat_lon

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Example

We will start by creating a weather forecast for the coming days. This will demonstrate how easy it is to run custom weather forecasts with [Earth2Studio](https://github.com/NVIDIA/earth2studio). Afterwards, we will dig into the details. For now, all we need is four lines of code to define the model, the data source, the IO backend, and finally, to kick off the workflow.

In [ ]:
model = SFNO.load_model(SFNO.load_default_package())
gfs = GFS()
io = KVBackend()
io = run.deterministic(["2025-03-03"], 16, model, gfs, io)

Let's hope for good weather! The animation below shows the temperature at a 2-meter height around San José (`t2m`).

In [ ]:
ds = io.to_xarray()
animate_local(ds.t2m, ds.time[0], *get_lat_lon("San Jose"), cb_label="Temperature [K]")

## Data sources

To understand what is going on, let's get familiar with some data sources and data handling tools. Earth observation data is complex and can be huge, so it is important to use effective tools. Earth2Studio provides a unified interface to different data repositories, which makes our lives easier. Our first step will be to explore ECMWF ERA5 data provided through the NSF NCAR mirror. Let’s start by creating our data source. The `NCAR_ERA5` data source provides access to ERA5 data from the NCAR mirror.

In [ ]:
ncar_era5 = NCAR_ERA5(verbose=True)

Because different datasets and data providers use different identifiers for the same variables, Earth2Studio provides lexicons to map between these identifiers. We can look at `NCAR_ERA5Lexicon` to find out which variables are available through `NCAR_ERA5`. Earth2Studio keeps these lexicons so you can use the same variable names across different data sources. Variables available on several different pressure levels are named with a single letter followed by the pressure level (e.g., `z500`, `q850`). Variables on single levels often follow the same naming scheme (e.g., `t2m`, `u10m`) but may also deviate (e.g., `tcwv`). The variable names generally match the short names in the [ECMWF parameter database](https://codes.ecmwf.int/grib/param-db/), with some exceptions.

In [ ]:
lexicon = NCAR_ERA5Lexicon()
print(list(lexicon.VOCAB))  # list of ERA5 variables

We can retrieve data by telling the data source which times and variables we are interested in. Below, we are selecting two dates in March 2023 and June 2024, and four variables: 2-meter temperature (`t2m`), total column water vapor (`tcwv`), and the u- and v-components of the 10-meter wind speed (`u10m`, `v10m`). Earth2Studio handles all the communication with the remote server under the hood and keeps a local cache. The data source returns an xarray `DataArray`, which we can convert to an xarray `Dataset` for convenience. 

In [ ]:
times = to_time_array(["2023-03-24 12:00:00", "2024-06-17 18:00:00"])
variables = ["t2m", "tcwv", "u10m", "v10m"]

ds_ncar_era5 = ncar_era5(times, variables).to_dataset("variable")

The `Dataset` contains the data for both time steps and all four variables. Let's plot total column water vapor (`tcwv`), which exhibits some nice swirls. The `plot.py` file supplied with this course contains some predefined plotting functionality to make exploring the data we will be dealing with more fun. The coastlines in the figures are provided by [cartopy](https://scitools.org.uk/cartopy/docs/latest/), a popular package for creating geospatial visualizations. The plotting function also allows you to choose a viewing angle in the form of central latitude and longitude. You can use `get_lat_lon` from `utils.py` for this, which provides coordinates for some cities across the globe (use `get_locations` to see which). You can also choose your own coordinates, of course.

In [ ]:
figure_global(ds_ncar_era5.isel(time=0).tcwv, *get_lat_lon("Cape Town"))

Next, we will load data for the same times and variables from the GFS analysis repository provided by NOAA/NCEP. Earth2Studio provides consistent interfaces and variable mappings, so all we need to do is change the data source to `GFS`.

In [ ]:
gfs = GFS()

ds_gfs = gfs(times, variables).to_dataset("variable")  # same as above

At first glance, the data looks similar, but you will be able to make out some differences when you compare the details. The GFS analysis repository is updated shortly after the data is available, while the ERA5 reanalysis is updated with a lag time of approximately 5 days. This makes GFS more suited for live forecasts. ERA5, on the other hand, provides consistent data spanning several decades, making it ideal for training global AI weather models and analyzing past weather events. For example, running forecasts on historical data (also referred to as hindcasts) can produce physically realistic event sets of counterfactual tropical cyclones used for calibrating risk models.

In [ ]:
figure_global(ds_gfs.isel(time=0).tcwv, *get_lat_lon("Cape Town"))

## Models

Our forecasts will be carried out by FourCastNet, a global AI weather model based on Spherical Fourier Neural Operators (SFNO). This model architecture allows learning long-range spatial dependencies and leveraging the symmetry of spherical systems like Earth.

The following figure, taken from [Li et al., 2021](https://arxiv.org/abs/2010.08895), shows the structure of a Fourier Neural Operator. Input `a(x)` is projected to a latent space representation (via `P`), processed by several Fourier layers, and projected back (via `Q`) to obtain the output `u(x)`. Check out the publications on [FNO](https://arxiv.org/abs/2010.08895) and [SFNO](https://arxiv.org/abs/2306.03838) to learn more.  

![FNO architecture](./images/fno.jpg "FNO architecture")

Earth2Studio provides pretrained models in the form of packages containing the model weights and additional parameters through `load_default_package`. These packages can then be loaded via `load_model`. Custom models can be trained with the [Modulus framework](https://github.com/NVIDIA/modulus), which focuses on allowing scaled training of physics-ML models.

In [ ]:
model = SFNO.load_model(SFNO.load_default_package())

Models describe their input and output interface via coords. Each coord corresponds to one dimension of the input and output tensor. The leading dimension is the batch dimension. The following dimensions correspond to the start time, the lead time within a forecast, the variables, and lastly, the geographic coordinates. The pretrained version of FourCastNet (SFNO) works with 73 atmospheric variables. Since the input and output variables are the same, we can feed each output back into the model to produce the next time step. The interval between time steps is 6 hours.

In [ ]:
print("Input coordinates: ", list(model.input_coords()))
print("Input variables:   ", list(model.input_coords()["variable"]))
print()
print("Output coordinates:", list(model.output_coords(model.input_coords())))
print("Output variables:  ", list(model.output_coords(model.input_coords())["variable"]))

## Perturbations

Weather forecasts deal with two main sources of uncertainty: Measurements and models. Meteorological measurements arrive from many different instruments located on ships, buoys, aircraft, satellites, weather stations, weather balloons, and others. Some are less accurate, some more accurate, but there will always be at least a tiny error associated with each measurement. This imperfect data is then assimilated through a model that is a close but not a perfect representation of reality. The same is true for the forecast model consuming the analysis. For example, small-scale processes are only modeled through approximate parameterizations. To capture these uncertainties, we need to run a probabilistic forecast, and the established method for this are ensemble forecasts.

To create an ensemble forecast, we run the same simulation for the same time horizon multiple times but with varying conditions. At the least, this means using differently perturbed initial conditions for every forecast run. In doing so, we take the uncertainty of the measurements and assimilation into account. Additionally, we can estimate the forecast model uncertainty by varying the model weights or using an inherently probabilistic model, e.g., based on diffusion. For this workshop, we will stick to initial condition perturbation and keep the weights fixed. For a scientifically more rigorous setup, we recommend the two-part publication by researchers from LBNL and NVIDIA on generating huge ensembles ([part 1](https://arxiv.org/abs/2408.03100) and [part 2]( https://arxiv.org/abs/2408.01581v1)).

Earth2Studio provides several perturbation methods. For example, the `SphericalGaussian` and `Brown` perturbations add spatially correlated noise to the input. Both methods accept a parameter to determine the noise amplitude and additional parameters to define the shape of the noise. The plot below illustrates different values for `SphericalGaussian`'s `alpha` and `Brown`'s `reddening` parameters, both making the noise smoother for higher values. Choosing the right perturbation method and parameters is application-dependent and needs to be validated as discussed in the second part of this workshop.

In [ ]:
plot_perturbations([1.0, 1.5, 2., 2.5], lats=121, lons=240)

For our ensemble forecast, we will apply perturbations through `SphericalGaussian` with a noise amplitude proportional to the mean value of each variable (`model.center`).

In [ ]:
sg = SphericalGaussian(noise_amplitude=0.03 * model.center[None, None].to("cuda"))

## Inference

Weather forecasts produce a lot of data, so an efficient IO process is vital. Earth2Studio provides IO backends for different file types and keeps track of variables, dimensions, and metadata. We will use an in-memory key-value store that we will write to a NetCDF4 archive once the forecast has finished. Later, we can load and analyze the data with the tool of our choice, for example, xarray.

In [ ]:
io = KVBackend()

It is now time to define some parameters for our first ensemble forecast. We want to start the forecast on July 2, 2024 (`start_time`), shortly before Hurricane Beryl made landfall on the Yucatán Peninsula and on the Southern US coast. Our forecast will produce 16 steps (`nsteps`), with 6-hour intervals in between. It will therefore span 4 days. We will start the forecast from 16 differently perturbed initial conditions (`nensemble`). FourCastNet (SFNO) produces a total of 73 variables, including temperature, wind speeds, geopotential height, humidity, and other meteorological factors. Your application may only depend on some of these variables, so Earth2Studio allows you to specify a set of output variables through the `output_coords` parameter. Only variables listed there will be saved to disk, reducing the size of the output considerably. Remember that with 721x1440 latitude/longitude data points per time slice, a single uncompressed 32-bit variable consumes approximately 4 MB of disk space. With many variables, long time horizons, and ensemble replications, this can add up quickly. The same `output_coords` parameter can also be used to only save a part of the latitude/longitude grid, for example.

We will request `t2m` (2-meter temperature), `u10m` (u-component of the wind, which is wind coming from the West, at a 10-meter height), `u100m` (u-component of the wind at a 100-meter height), `v10m` (v-component of the wind, coming from the South, at a 10-meter height), `v100m` (v-component of the wind at a 100-meter height), `tcwv` (total column vertically integrated water vapor), and `z500` (geopotential height at 500 hPa). To be clear, all 73 variables for the forecast will be produced because they are needed for the successive forecast step. But only these 7 variables will end up on disk.

In [ ]:
start_time = np.datetime64("2024-07-04 00:00:00")
nsteps = 32
nensemble = 16
batch_size = 16
output_coords = {"variable": np.array(["t2m", "u10m", "v10m", "u100m", "v100m", "tcwv", "z500"])}

Earth2Studio provides predefined workflows for easy-to-run ensemble forecasts and other use cases. We simply pass the parameters defined above. The workflow fetches the initial conditions, sets up the IO store, applies the perturbation, and finally runs the forecast.

In [ ]:
io = run.ensemble(
    [start_time],
    nsteps=nsteps,
    nensemble=nensemble,
    prognostic=model,
    data=ncar_era5,
    io=io,
    perturbation=sg,
    batch_size=batch_size,
    output_coords=output_coords,
)

We can explore the output data with xarray, which provides some convenient data handling functionalities.

In [ ]:
!mkdir -p /workspace/exercises/output && rm -f /workspace/exercises/output/part_01.nc
ds = io.to_xarray()
ds.to_netcdf("/workspace/exercises/output/part_01.nc")  # save for part 2

In [ ]:
ds

The resulting dataset has five coordinates:

1. `ensemble`, with one forecast for each of the 16 ensemble members
2. `time`, which is our start time
3. `lead_time`, with 33 values made up of the initial conditions and the 32 forecast steps
4. `lat`, with 721 latitude values
5. `lon`, with 1440 longitude values

As specified, the dataset contains seven variables: `t2m`, `u10m`, `v10m`, `u100m`, `v100m`, `tcwv`, and `z500`. We will conclude this part of the workshop with a simple plot and continue exploring the data in the next part.

In [ ]:
figure_global(ds.isel(time=0, lead_time=3, ensemble=0).tcwv, *get_lat_lon("San Jose"))

When you are done with this notebook, please shut down the kernel (in the context menu at the top, choose "Kernel" and then "Shut Down Kernel"). This will unload the model and all tensors from the GPU and make sure we can start fresh in the following exercises. Then proceed to the [notebook for part 2](./chapter_04_ensemble_forecasting.ipynb), where we will inspect our forecast in more detail.

## Extra

For the example above, we have used a rather simple spherical Gaussian perturbation. Researchers from LBNL and NVIDIA developed a perturbation method specifically for global AI forecasting (see [part 1](https://arxiv.org/abs/2408.03100) and [part 2]( https://arxiv.org/abs/2408.01581v1) of the publication also linked above). Their perturbation method, together with inference across multiple independently trained model checkpoints, was key to the first ever scientifically sound evaluation of large AI ensembles. If you have extra time on your hands, you can follow the instructions below to set up the perturbation method (we will not be going into running inference across multiple checkpoints here).

The objective of a good perturbation method is to provide a realistic measure of uncertainty for each variable. One common approach for this is bred vectors, which capture the instabilities of a system. Fast-moving variables are perturbed more than slow-moving variables. The perturbation strength is determined by comparing the difference between a forecast without perturbation and a forecast with a simpler seeding perturbation. Bred vector perturbation therefore already uses our model before the start of the actual forecast. The method used below also applies specific rescaling separately for each hemisphere.

In [ ]:
skill_path = "./data/sfno_linear_skill_48h.nc"
noise_amplitude_seed = get_noise_vector(
    model,
    skill_path=skill_path,
    noise_amplification=0.35,
    vars=["z500"],
)
noise_amplitude_iter = get_noise_vector(
    model,
    skill_path=skill_path,
    noise_amplification=0.35,
)
seeding_perturbation = CorrelatedSphericalGaussian(noise_amplitude=noise_amplitude_seed)

hcbv = HemisphericCentredBredVector(
    model=model,
    data=ncar_era5,
    time=start_time,
    noise_amplitude=noise_amplitude_iter,
    integration_steps=3,
    seeding_perturbation_method=seeding_perturbation,
)

Once we have set up `HemisphericCentredBredVector`, we can simply pass it to the same Earth2Studio workflow as above. 

In [ ]:
io_extra = KVBackend()
io_extra = run.ensemble(
    [start_time],
    nsteps=nsteps,
    nensemble=nensemble,
    prognostic=model,
    data=ncar_era5,
    io=io_extra,
    perturbation=hcbv,
    batch_size=batch_size,
    output_coords=output_coords,
)
!mkdir -p /workspace/exercises/output && rm -f /workspace/exercises/output/part_01_extra.nc
ds_extra = io_extra.to_xarray()
ds_extra.to_netcdf("/workspace/exercises/output/part_01_extra.nc")

Let's have a quick look at the results.

In [ ]:
plot_global_ensemble(ds_extra.tcwv, start_time, *get_lat_lon("Tokyo"), cb_label="tcwv [kg/m²]", ilead_time=10)